In [2]:
import os

os.chdir('..')

#
os.chdir('Datasets')
print("Current Working Directory " , os.getcwd())

Current Working Directory  E:\MITB\Text Analytics\Project\Datasets\Submission_folder\Datasets


In [3]:
import pandas as pd
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()
import gensim
from gensim import corpora,similarities
from nltk.corpus import stopwords
import pprint
stop_list=stopwords.words('english')
import sys

In [9]:
# Read files

topdocs = pd.read_csv("./Dominant Topic for each Document.csv")
topiclist = pd.read_csv("./doc_topic_mixture.csv")
data = pd.read_csv("./Songs_clean_en.csv")
data['Documents'] = data.index
data=data[['Documents','Band','Lyrics']]
top_words=pd.read_csv("./top_words.csv")
top_words=top_words[['Topic','Word','P']]

print(topdocs.head())
print(topiclist.head())
print(data.head())
print(top_words.head())

   Unnamed: 0  Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0           0            0             4.0              0.4837   
1           1            1             7.0              0.3897   
2           2            2             4.0              0.6420   
3           3            3             1.0              0.4912   
4           4            4             4.0              0.3067   

                                            Keywords  \
0  love, like, see, heart, away, night, time, eye...   
1  like, girl, know, good, could, say, little, go...   
2  love, like, see, heart, away, night, time, eye...   
3  love, know, baby, want, let, go, never, got, s...   
4  love, like, see, heart, away, night, time, eye...   

                                                Text  
0  ['drinks', 'go', 'smoke', 'goes', 'feel', 'got...  
1  ['trippin', 'grigio', 'mobbin', 'lights', 'low...  
2  ['see', 'midnight', 'panther', 'gallant', 'bra...  
3  ['believe', 'believe', 'question', 'takes',

In [10]:
topicwords = topdocs['Keywords'].unique()
topicwords=topicwords.tolist()
words=[]

for i in range(len(topicwords)):
    words.append(nltk.word_tokenize(topicwords[i])) 
    
new_words=[]
for i in range(len(words)):
    new_words.append(list(set(words[i])))
    
for i in range(len(new_words)):
    new_words[i].sort()
    new_words[i].pop(0)

dictionary = corpora.Dictionary(new_words)
print("Final topics are :",new_words)

Final topics are : [['away', 'day', 'eyes', 'heart', 'like', 'love', 'night', 'one', 'see', 'time'], ['could', 'girl', 'good', 'got', 'know', 'like', 'little', 'say', 'think', 'time'], ['baby', 'go', 'got', 'know', 'let', 'love', 'need', 'never', 'say', 'want'], ['baby', 'back', 'come', 'get', 'go', 'gon', 'got', 'know', 'let', 'like'], ['blood', 'dead', 'death', 'die', 'god', 'life', 'one', 'people', 'us', 'world'], ['came', 'home', 'lord', 'man', 'old', 'one', 'said', 'town', 'well', 'went'], ['blue', 'christmas', 'happy', 'jesus', 'little', 'mary', 'snow', 'sweet', 'train', 'tree'], ['ass', 'bitch', 'cause', 'fuck', 'get', 'know', 'like', 'nigga', 'niggas', 'shit']]


In [11]:
# Create Dictionary
token_to_id = dictionary.token2id
print(type(token_to_id))
print(token_to_id)

# Converting the document into vectors
vecs=[dictionary.doc2bow(doc) for doc in new_words]

# Creating similarity matrix
index = similarities.SparseMatrixSimilarity(vecs,62)

<class 'dict'>
{'away': 0, 'day': 1, 'eyes': 2, 'heart': 3, 'like': 4, 'love': 5, 'night': 6, 'one': 7, 'see': 8, 'time': 9, 'could': 10, 'girl': 11, 'good': 12, 'got': 13, 'know': 14, 'little': 15, 'say': 16, 'think': 17, 'baby': 18, 'go': 19, 'let': 20, 'need': 21, 'never': 22, 'want': 23, 'back': 24, 'come': 25, 'get': 26, 'gon': 27, 'blood': 28, 'dead': 29, 'death': 30, 'die': 31, 'god': 32, 'life': 33, 'people': 34, 'us': 35, 'world': 36, 'came': 37, 'home': 38, 'lord': 39, 'man': 40, 'old': 41, 'said': 42, 'town': 43, 'well': 44, 'went': 45, 'blue': 46, 'christmas': 47, 'happy': 48, 'jesus': 49, 'mary': 50, 'snow': 51, 'sweet': 52, 'train': 53, 'tree': 54, 'ass': 55, 'bitch': 56, 'cause': 57, 'fuck': 58, 'nigga': 59, 'niggas': 60, 'shit': 61}


In [13]:
query=input("Enter the query: ")

Enter the query: I am happy happy


In [14]:
# Query Processing
query_lower=query.lower()
query_words=nltk.word_tokenize(query_lower)
query_alpha=[w for w in query_words if re.search('^[a-z]+$',w)]
query_stop=[w for w in query_alpha if w not in stop_list]

In [15]:
try:
    query_vec=[dictionary.doc2bow(doc) for doc in query_stop]
    similarity=index[query_vec]
    #print(list(enumerate(similarity)))
    sorted_similarity = sorted(enumerate(similarity), key = lambda item: -item[1])
    topic_list = sorted_similarity[0][0]

except TypeError:
    topic_list=[]
    for i in range(len(new_words)):
        if (query_stop[0] in new_words[i]):
            topic_list.append(i)
            continue

In [16]:
if(len(topic_list)==0):
    print("Sorry! We could not narrow down the topics, please enter a different song. Thank you!")
    sys.exit()
    
if (len(topic_list) == 1):
    print("The narrowed topic list is:",topic_list)
    
if(len(topic_list) == 2):
    print("We have multiple topics from the topic word distribution")
    print("The narrowed topic list is: Topic {0:d} and Topic {1:d}".format(topic_list[0], topic_list[1]))
    
if(len(topic_list) == 3):
    print("We have multiple topics from the topic word distribution")
    print("The narrowed topic list is: Topic {0:d}, Topic {1:d} and Topic {2:d}".format(topic_list[0], topic_list[1],topic_list[2]))

if(len(topic_list)>3):
    print("The topic list limit has exceeded")

The narrowed topic list is: [6]


In [17]:
query_stop

['happy', 'happy']

In [18]:
#topic_list =[2,3]
# Function to retrieve the final list of documents
def retrieve_list(out):
    if (len(out)==1):
        num=str(out[0]+1)
        final_list=topiclist[num].dropna().sort_values(ascending=False).head().to_frame()
        final_list.columns=['Topic '+str(topic_list[0])]
        final_list['Documents']=final_list.index
        name1,name2=final_list.columns
        final_list=final_list[[name2,name1]]
        return final_list
    
    if (len(out)>1):
        num1=str(out[0]+1)
        num2=str(out[1]+1)
        final_list1=topiclist[num1].dropna().sort_values(ascending=False).head().to_frame()
        final_list2=topiclist[num2].dropna().sort_values(ascending=False).head().to_frame()
        final_list1.columns=['Topic '+str(topic_list[0])]
        final_list2.columns=['Topic '+str(topic_list[1])]
        final_list1['Documents']=final_list1.index
        final_list2['Documents']=final_list2.index
        name1,name2=final_list1.columns
        name3,name4=final_list2.columns
        final_list1=final_list1[[name2,name1]]
        final_list2=final_list2[[name4,name3]]
        return final_list1,final_list2


In [25]:
show=retrieve_list(topic_list)
print(show)

        Documents   Topic 6
46695       46695  0.985854
20371       20371  0.983771
77326       77326  0.983764
106020     106020  0.983147
70860       70860  0.983146


pandas.core.frame.DataFrame

In [26]:
# Code for document retrieval from the topic calculated if single topic is retrieved

if (isinstance(show,pd.DataFrame)):
    #print(show)
    doc_list=list(show['Documents'])
    #print(doc_list)
    last=data[data['Documents'] .isin(doc_list)]
    last=last[['Band','Lyrics']]
    last['Document']=last.index
    print(last)
    
#Dual topic resolution and document retrieval for the topic
if (isinstance(show,tuple)):
    print(show[0])
    print(show[1])
    frame=pd.DataFrame()
    for i in range(len(query_stop)):
        frame=top_words[top_words['Word'].isin(query_stop)]
    print(frame)
    frame=frame.groupby('Topic').sum()
    frame['Topic']=frame.index
    p=frame[frame['Topic'].isin(topic_list)]
    topic=p.sort_values('P').head(1)['Topic'].to_list()
    print("Sorting is done. And the final topic is",topic)
    show=retrieve_list(topic)
    print("This is from Dual topic resolution and document retrieval for the topic block :",show)
    
    #final_dual_topic=frame.sort_values('P',ascending=False).head(1)['Topic']
    #print("The final topic selected is ",final_dual_topic)
    #doc_list=final_dual_topic.to_list()
    #print("The final topic selected is ",doc_list[0])
    #last=last[['Band','Lyrics']]
    #print(last)
    #last=data[data['Documents'].isin(doc_list)]
    #print(last)
    #last=last[['Band','Lyrics']]
    #print(last)

                      Band                                             Lyrics  \
20371             Deftones  Decide, decide\r\r\r\nIs this safe to say?\r\r...   
46695       Aselin Debison  Hmm hmm hmm\r\r\r\nFriends fight about stupid ...   
70860         Dolly Parton  Happy, happy birthday baby \r\r\r\nAlthough yo...   
77326   Ocean Colour Scene  You're lining your pockets\r\r\nFor no other r...   
106020        Dolly Parton  A cross between a movie star \r\r\r\nAnd a her...   

        Document  
20371      20371  
46695      46695  
70860      70860  
77326      77326  
106020    106020  


In [28]:
# Code for document retrieval from the topic calculated if single topic is retrieved

if (isinstance(show,pd.DataFrame)):
    #print(show)
    doc_list= list(show['Documents'])
    #print(doc_list)
    last=data[data['Documents'] .isin(doc_list)]
    last=last[['Band','Lyrics']]
    last['Document']=last.index
    print(last)

                      Band                                             Lyrics  \
20371             Deftones  Decide, decide\r\r\r\nIs this safe to say?\r\r...   
46695       Aselin Debison  Hmm hmm hmm\r\r\r\nFriends fight about stupid ...   
70860         Dolly Parton  Happy, happy birthday baby \r\r\r\nAlthough yo...   
77326   Ocean Colour Scene  You're lining your pockets\r\r\nFor no other r...   
106020        Dolly Parton  A cross between a movie star \r\r\r\nAnd a her...   

        Document  
20371      20371  
46695      46695  
70860      70860  
77326      77326  
106020    106020  
